In [32]:
import cv2
import numpy as np
from queue import Queue
import matplotlib.pyplot as plt
import random

MatLike = cv2.typing.MatLike

In [33]:
def showImage(img, name):
  cv2.imshow(name, img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [34]:
def readImage(imagePath: str) -> MatLike:
    return cv2.imread(imagePath)

In [35]:
def convertImageToBinary(image: MatLike) -> MatLike:
    grayScaleImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    showImage(grayScaleImage, 'GrayScale')

    _, binaryImage = cv2.threshold(grayScaleImage, 125, 255, cv2.THRESH_BINARY)
    showImage(binaryImage, 'Binary')

    return binaryImage


In [36]:
def labelBlackAndWhiteImage_traversaInLatime(image: MatLike) -> np.typing.NDArray:
    height, width = image.shape
    labels = np.zeros((height, width), dtype=int)
    label = 0

    for i in range(height):
        for j in range(width):
            if image[i, j] == 0 and labels[i, j] == 0:
                label += 1
                q = Queue()
                q.put((i, j))
                labels[i, j] = label

                while not q.empty():
                    x, y = q.get()
                    
                    neighbors = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
                    for nx, ny in neighbors:
                        if 0 <= nx < height and 0 <= ny < width:
                            if image[nx, ny] == 0 and labels[nx, ny] == 0:
                                q.put((nx, ny))
                                labels[nx, ny] = label

    return labels

In [37]:
def labelBlackAndWhiteImage_douaTreceriCuClaseDeEchivalenta(image: MatLike) -> np.typing.NDArray:
    height, width = image.shape
    labels = np.zeros((height, width), dtype=int)
    label = 0
    edges = {}

    neighbors = [(-1, 0), (0, -1), (-1, -1), (1, -1)]

    for i in range(height):
        for j in range(width):
            if image[i, j] == 0 and labels[i, j] == 0:
                l = set()
                for dx, dy in neighbors:
                    nx, ny = i + dx, j + dy
                    if 0 <= nx < height and 0 <= ny < width and labels[nx, ny] > 0:
                        l.add(labels[nx, ny])
                
                if not l:
                    label += 1
                    labels[i, j] = label
                else:
                    x = min(l)
                    labels[i, j] = x
                    for y in l:
                        if y != x:
                            edges.setdefault(x, set()).add(y)
                            edges.setdefault(y, set()).add(x)
    

    new_label = 0
    new_labels = np.zeros(label+1, dtype=int)
    
    for i in range(1, label):
        if new_labels[i] == 0:
            new_label += 1
            new_labels[i] = new_label
            Q = [i]
            while Q:
                x = Q.pop(0)
                for y in edges.get(x, []):
                    if new_labels[y] == 0:
                        new_labels[y] = new_label
                        Q.append(y)

    for i in range(height):
        for j in range(width):
            if labels[i, j] > 0:
                labels[i, j] = new_labels[labels[i, j]]

    return labels


In [38]:
def colorLabeledImage(image):
    if not isinstance(image, np.ndarray):
        raise ValueError("Input should be a numpy ndarray.")

    labels = np.unique(image)
    labels = labels[labels != 0]

    height, width = image.shape
    colored_image = np.zeros((height, width, 3), dtype=np.uint8)

    colors = {0: (255, 255, 255)}
    for label in labels:
        colors[label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    for label, color in colors.items():
        colored_image[image == label] = color

    return colored_image
                

In [39]:
def main():
    imagePath: str = "tickets.jpg"
    image: MatLike = readImage(imagePath)

    if image is None:
        print("Error reading image!")
        return 1

    blackAndWhiteImage: MatLike = convertImageToBinary(image)

    labeledImage = labelBlackAndWhiteImage_traversaInLatime(blackAndWhiteImage)
    coloredImage = colorLabeledImage(labeledImage)
    showImage(coloredImage, "Colored image - traversare in latime")

    labeledImage = labelBlackAndWhiteImage_douaTreceriCuClaseDeEchivalenta(blackAndWhiteImage)
    coloredImage = colorLabeledImage(labeledImage)
    showImage(coloredImage, "Colored image - doua treceri cu clase de echivalenta")
    
if __name__ == "__main__":
    main()